# Regresión

En los problemas de **regresión,** la salida es un real $y\in\mathbb{R}$ en lugar de una etiqueta de clase $y\in\{1,\dotsc,C\}$, como ocurre en clasificación. La función de pérdida usual en regresión es la **pérdida cuadrática** o **L2:**
$$\ell_2(y,\hat{y})=(y-\hat{y})^2$$
La diferencia entre observación y predicción, $y-\hat{y}$, recibe el nombre de **residuo.** Llamamos **error cuadrático medio (MSE, mean squared error)** al riesgo empírico con pérdida cuadrática:
$$\operatorname{MSE}(\boldsymbol{\theta})=\frac{1}{N}\sum_n (y_n-f(\boldsymbol{x}_n; \boldsymbol{\theta}))^2$$
El modelo probabilístico condicional que suele usarse para capturar la incertidumbre subyacente es la **distribución normal o Gaussiana condicional (a posteriori):**
$$p(y\mid \boldsymbol{x}; \boldsymbol{\theta})%
=\mathcal{N}(y\mid \mu,\sigma^2)=\frac{1}{\sqrt{2\pi\sigma^2}}\exp\left[{-\frac{1}{2\sigma^2}(y-\mu)^2}\right]%
\qquad\text{con}\qquad%
\mu=f(\boldsymbol{x}; \boldsymbol{\theta})$$
Nótese que la función predictora se limita a predecir la media (a posteriori); por simplicidad en este contexto, la varianza $\sigma^2$ se asume fija y conocida.